<img style='float: left' width="150px" src="http://bostonlightswim.org/wp/wp-content/uploads/2011/08/BLS-front_4-color.jpg">
<br><br>

## [The Boston Light Swim](http://bostonlightswim.org/)

### Sea Surface Temperature time-series model skill

### Load configuration

In [1]:
import os
import sys
import warnings

ioos_tools_path = os.path.join(os.path.pardir, os.path.pardir)
sys.path.append(ioos_tools_path)

# Suppresing warnings for a "pretty output."
# Remove this line to debug any possible issues.
warnings.simplefilter("ignore")

In [2]:
from ioos_tools.ioos import parse_config

config = parse_config('config.yaml')

save_dir = os.path.join(os.path.abspath(config['run_name']))

### Skill 1: Model Bias (or Mean Bias)

The bias skill compares the model mean temperature against the observations.
It is possible to introduce a Mean Bias in the model due to a mismatch of the
boundary forcing and the model interior.

$$ \text{MB} = \mathbf{\overline{m}} - \mathbf{\overline{o}}$$

In [3]:
from ioos_tools.ioos import stations_keys


def rename_cols(df, config):
    cols = stations_keys(config, key='station_name')
    return df.rename(columns=cols)

In [4]:
from ioos_tools.ioos import to_html, save_html, load_ncs
from ioos_tools.skill_score import mean_bias, apply_skill

dfs = load_ncs(config)

df = apply_skill(dfs, mean_bias, remove_mean=False, filter_tides=False)
skill_score = dict(mean_bias=df.to_dict())

# Filter out stations with no valid comparison.
df.dropna(how='all', axis=1, inplace=True)
df = df.applymap('{:.2f}'.format).replace('nan', '--')

# Pretty HTML table.
df = rename_cols(df, config)
html = to_html(df.T)
fname = os.path.join(save_dir, 'mean_bias.html')
save_html(fname, html)
html

,G1_SST_GLOBAL,NECOFS_FVCOM_OCEAN_MASSBAY_FORECAST,NECOFS_GOM3_FORECAST,coawst_4_use_best,global
"BOSTON 16 NM East of Boston, MA",-0.47,-0.48,-0.38,-1.71,-2.14
Buoy A0102 - Mass. Bay/Stellwagen,0.62,0.45,0.26,-1.24,-0.98
"Boston, MA",-1.05,0.86,0.78,-0.61,--


### Skill 2: Central Root Mean Squared Error

Root Mean Squared Error of the deviations from the mean.

$$ \text{CRMS} = \sqrt{\left(\mathbf{m'} - \mathbf{o'}\right)^2}$$

where: $\mathbf{m'} = \mathbf{m} - \mathbf{\overline{m}}$ and $\mathbf{o'} = \mathbf{o} - \mathbf{\overline{o}}$

In [5]:
from ioos_tools.skill_score import rmse

dfs = load_ncs(config)

df = apply_skill(dfs, rmse, remove_mean=True, filter_tides=False)
skill_score['rmse'] = df.to_dict()

# Filter out stations with no valid comparison.
df.dropna(how='all', axis=1, inplace=True)
df = df.applymap('{:.2f}'.format).replace('nan', '--')

# Pretty HTML table.
df = rename_cols(df, config)
html = to_html(df.T)
fname = os.path.join(save_dir, 'rmse.html')
save_html(fname, html)
html

,G1_SST_GLOBAL,NECOFS_FVCOM_OCEAN_MASSBAY_FORECAST,NECOFS_GOM3_FORECAST,coawst_4_use_best,global
"BOSTON 16 NM East of Boston, MA",0.48,0.52,0.44,0.52,0.38
Buoy A0102 - Mass. Bay/Stellwagen,0.90,0.55,0.33,0.81,0.91
"Boston, MA",0.76,0.37,0.41,0.72,--


### Skill 3: R$^2$
https://en.wikipedia.org/wiki/Coefficient_of_determination

In [6]:
from ioos_tools.skill_score import r2

dfs = load_ncs(config)

df = apply_skill(dfs, r2, remove_mean=True, filter_tides=False)
skill_score['r2'] = df.to_dict()

# Filter out stations with no valid comparison.
df.dropna(how='all', axis=1, inplace=True)
df = df.applymap('{:.2f}'.format).replace('nan', '--')

# Pretty HTML table.
df = rename_cols(df, config)
html = to_html(df.T)
fname = os.path.join(save_dir, 'r2.html')
save_html(fname, html)
html

,G1_SST_GLOBAL,NECOFS_FVCOM_OCEAN_MASSBAY_FORECAST,NECOFS_GOM3_FORECAST,coawst_4_use_best,global
"BOSTON 16 NM East of Boston, MA",0.59,-0.81,-0.32,0.57,0.74
Buoy A0102 - Mass. Bay/Stellwagen,0.63,-1.41,0.14,0.35,0.62
"Boston, MA",-0.38,0.42,0.30,-0.82,--


In [7]:
import json

fname = os.path.join(save_dir, 'skill_score.json')

# Stringfy keys for json.
for key in skill_score.keys():
    skill_score[key] = {str(k): v for k, v in skill_score[key].items()}

with open(fname, 'w') as f:
    f.write(json.dumps(skill_score))

### Normalized Taylor diagrams

The radius is model standard deviation error divided  by observations deviation,
azimuth is arc-cosine of cross correlation (R), and distance to point (1, 0) on the
abscissa is Centered RMS.

In [8]:
%matplotlib inline
import matplotlib.pyplot as plt
from ioos_tools.taylor_diagram import TaylorDiagram


def make_taylor(samples):
    fig = plt.figure(figsize=(9, 9))
    dia = TaylorDiagram(samples['std']['OBS_DATA'],
                        fig=fig,
                        label="Observation")
    colors = plt.matplotlib.cm.jet(np.linspace(0, 1, len(samples)))
    # Add samples to Taylor diagram.
    samples.drop('OBS_DATA', inplace=True)
    for model, row in samples.iterrows():
        dia.add_sample(row['std'], row['corr'], marker='s', ls='',
                       label=model)
    # Add RMS contours, and label them.
    contours = dia.add_contours(colors='0.5')
    plt.clabel(contours, inline=1, fontsize=10)
    # Add a figure legend.
    kw = dict(prop=dict(size='small'), loc='upper right')
    leg = fig.legend(dia.samplePoints,
                     [p.get_label() for p in dia.samplePoints],
                     numpoints=1, **kw)
    return fig

In [9]:
import numpy as np
import pandas as pd

dfs = load_ncs(config)

# Bin and interpolate all series.
freq = '30min'
for station, df in list(dfs.iteritems()):
    df = df.resample(freq).interpolate().dropna(axis=1)
    if 'OBS_DATA' in df:
        samples = pd.DataFrame.from_dict(dict(std=df.std(),
                                              corr=df.corr()['OBS_DATA']))
    else:
        continue
    samples[samples < 0] = np.NaN
    samples.dropna(inplace=True)
    if len(samples) <= 2:  # 1 obs 1 model.
        continue
    fig = make_taylor(samples)
    fig.savefig(os.path.join(save_dir, '{}.png'.format(station)))
    plt.close(fig)